# **Método EffectiveNet** #

En este notebook se trabajará el método de EffectiveNet, ya que por problemas de espacio no se pudo llevar a cabo en el notebook en donde se hizo el proyecto. Pero de igual manera se desarrollará paso a paso para que se pueda observar la metodología y resultados. La comparación con el método tradicional se discutirá en el notebook principal.

## **Metodología** ##

La metodología en la que se trbajará el método de EffectiveNet será similar a la metodología usada en el método de Redes Neuronales, se evaluará un modelo preliminar (con el val test) y luego se modificará el modelo principal para ajustar, si es necesario hacer ciertos ajustes y se evaluará con el test set.

In [1]:
# Librerías normales #
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Librerías relacionadas con las Redes Neuronales #
import tensorflow as tf
print(tf.__version__)
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras.callbacks import TensorBoard 
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split

%matplotlib inline

#Librerías EfficientNet03
from keras import applications
from tensorflow.keras.applications import EfficientNetB3

2.4.1


In [2]:
# En primer lugar verificamos que sí estemos usando un GPU para que corra más rápido el script. #
# (Usamos COLAB ya que la computadora no tenía la capacidad para correr el código usando el CPU normal) #
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# Observamos los devices que se tienen disponibles #
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 361200605443397075, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5110844379384280527
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [4]:
# Con esta función podemos "traer" los datos que subimos a drive para poder usarlos #
# (Se necesita hacer login en la cuenta para acceder a todos los datos, pero estos se pueden #
# enviar si se desean) #
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Se reduce el tamaño de las imágenes ya que si se dejaba cómo estaba la página (Colab) #
# No soportaba el tamaño y se quedaba sin memoria para poder correr el modelo. #

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


traindata2 = image_generator.flow_from_directory("/content/gdrive/My Drive/xray_pulmones/train",
                                           target_size = (50, 50),
                                           class_mode = 'categorical',
                                           batch_size = 500)
testdata2 = image_generator.flow_from_directory('/content/gdrive/My Drive/xray_pulmones/test',
                                           target_size = (50, 50),
                                           class_mode = 'categorical',
                                           batch_size = 75)
valdata2 = image_generator.flow_from_directory('/content/gdrive/My Drive/xray_pulmones/val',
                                           target_size = (50, 50),
                                           class_mode = 'categorical',
                                           batch_size = 75)

Found 1620 images belonging to 2 classes.
Found 493 images belonging to 2 classes.
Found 330 images belonging to 2 classes.


In [6]:
# Previamente se impoortó la librería y a continuación se "armará" el modelo. #
# Primero cargamos el modelo y primero definimos algunas características, #
# como include_top que define si usamos la capa completa y la conectamos con la siguiente. #

eff_net = EfficientNetB3
eff_net_modelo = eff_net(include_top = False, input_shape = (50, 50, 3), pooling = 'max')

modelo2 = Sequential()
modelo2.add(eff_net_modelo)
modelo2.add(Dense(units = 32, activation = 'relu'))
modelo2.add(Dense(units = 64, activation = 'relu'))
modelo2.add(Dropout(0.25))
modelo2.add(Dense(units = 2, activation= 'softmax'))

modelo2.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

43941888/43941136 [==============================] - 0s 0us/step


In [8]:
resultado_EN3_1 = modelo2.fit_generator(traindata2, epochs = 2, validation_data = valdata2)

Epoch 1/2
4/4 [==============================] - 493s 138s/step - loss: 0.7966 - accuracy: 0.5554 - val_loss: 0.7106 - val_accuracy: 0.5000
Epoch 2/2
4/4 [==============================] - 26s 6s/step - loss: 0.4147 - accuracy: 0.8283 - val_loss: 0.7178 - val_accuracy: 0.5000


In [9]:
resultado_EN3_2 = modelo2.fit_generator(traindata2, epochs = 10, validation_data = valdata2)

Epoch 1/10
4/4 [==============================] - 27s 7s/step - loss: 0.2028 - accuracy: 0.9228 - val_loss: 0.8353 - val_accuracy: 0.5000
Epoch 2/10
4/4 [==============================] - 26s 6s/step - loss: 0.1434 - accuracy: 0.9444 - val_loss: 1.5690 - val_accuracy: 0.5000
Epoch 3/10
4/4 [==============================] - 26s 6s/step - loss: 0.0974 - accuracy: 0.9593 - val_loss: 2.1521 - val_accuracy: 0.5000
Epoch 4/10
4/4 [==============================] - 26s 6s/step - loss: 0.0747 - accuracy: 0.9728 - val_loss: 2.7453 - val_accuracy: 0.5000
Epoch 5/10
4/4 [==============================] - 26s 6s/step - loss: 0.0656 - accuracy: 0.9759 - val_loss: 4.1348 - val_accuracy: 0.5000
Epoch 6/10
4/4 [==============================] - 26s 6s/step - loss: 0.0447 - accuracy: 0.9821 - val_loss: 5.4078 - val_accuracy: 0.5000
Epoch 7/10
4/4 [==============================] - 25s 6s/step - loss: 0.0593 - accuracy: 0.9790 - val_loss: 5.7862 - val_accuracy: 0.5000
Epoch 8/10
4/4 [==================

Observamos que el modelo logra clasificar las imágenes de una manera excelente. Se llevará a cabo una última prueba con un drop más alto para terminar de confirmar que no hay overfitting.

In [10]:
eff_net = EfficientNetB3
eff_net_modelo = eff_net(include_top = False, input_shape = (50, 50, 3), pooling = 'max')

modelo2 = Sequential()
modelo2.add(eff_net_modelo)
modelo2.add(Dense(units = 32, activation = 'relu'))
modelo2.add(Dropout(0.25))
modelo2.add(Dense(units = 64, activation = 'relu'))
modelo2.add(Dropout(0.25))
modelo2.add(Dense(units = 2, activation= 'softmax'))

modelo2.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
resultado_EN3_4 = modelo2.fit_generator(traindata2, epochs = 10, validation_data = testdata2)

Epoch 1/10
4/4 [==============================] - 41s 10s/step - loss: 0.3715 - accuracy: 0.8389 - val_loss: 0.6518 - val_accuracy: 0.6653
Epoch 2/10
4/4 [==============================] - 29s 9s/step - loss: 0.2044 - accuracy: 0.9265 - val_loss: 0.6824 - val_accuracy: 0.6653
Epoch 3/10
4/4 [==============================] - 29s 7s/step - loss: 0.1248 - accuracy: 0.9556 - val_loss: 0.8061 - val_accuracy: 0.3347
Epoch 4/10
4/4 [==============================] - 29s 7s/step - loss: 0.0860 - accuracy: 0.9660 - val_loss: 1.2928 - val_accuracy: 0.3347
Epoch 5/10
4/4 [==============================] - 29s 9s/step - loss: 0.0668 - accuracy: 0.9778 - val_loss: 1.7313 - val_accuracy: 0.3347
Epoch 6/10
4/4 [==============================] - 29s 9s/step - loss: 0.0410 - accuracy: 0.9852 - val_loss: 2.0807 - val_accuracy: 0.3347
Epoch 7/10
4/4 [==============================] - 28s 7s/step - loss: 0.0317 - accuracy: 0.9889 - val_loss: 2.4966 - val_accuracy: 0.3347
Epoch 8/10
4/4 [=================

Podemos observar que el predictive score del modelo es bastante alto y la comparación con el primer modelo se hará en el notebook principal.